In [ ]:
import os
import cv2
import math
import pandas as pd
import numpy as np
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import matplotlib.cm as cmx
import matplotlib.colors as cl
from matplotlib import transforms
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from scipy import signal

In [ ]:
os.getcwd()
os.chdir("C:/example")

In [ ]:
width=1920; height=1080; FPS=2
factor=10

Evolution=1000

position_vert=np.zeros((Evolution)); position_hort=np.zeros((Evolution))

f=open("exampleTrackTable.txt",'w+')
cap=cv2.VideoCapture("exampleVideo.mp4")
i=0; j=0; k=0;

while ((cap.isOpened()) and (k < Evolution)):
    tempo_step=i+0
    cap.set(1, (tempo_step))
    ret,frame=cap.read()
    
    if ret == False:
        break

    z=1
    k=k+1
    maior_horizontal=0; maior_vertical=0
    for l in range(int(0*height/6),int(6*height/6),factor):
        for ll in range(int(0*width/10),int(10*width/10),factor):
            Quant=0
            for lll in range(0,factor):
                for llll in range(0,factor):
                    if ((frame[lll+l,llll+ll,2] > 150) and (frame[lll+l,llll+ll,1] < 100) and (frame[lll+l,llll+ll,0] < 100)):
                        Quant=Quant+1
            if (Quant > (factor*factor*0.2)):
                position_vert[z-1]=l+(factor/2)
                position_hort[z-1]=ll+(factor/2)
                if (position_hort[z-1] > maior_horizontal):
                    maior_horizontal=position_hort[z-1]
                    maior_vertical=position_vert[z-1]
                z=z+1

    f.write('%d %d %d\n' %(tempo_step,maior_vertical,maior_horizontal))
    print(k)
    i+=2

cap.release()
cv2.destroyAllWindows()


In [ ]:
plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
params = {'text.latex.preamble' : [r'\usepackage{siunitx}', r'\usepackage{amsmath}']}
plt.rcParams.update(params)

Data_Number = 59

for i in range (1,Data_Number ):
    string_vector=[a]
    
    a1=np.loadtxt(a[i])
    a1_N=len(a1)
    freq=15; win=60; overlap=40
    
    Data_Number=1
    
    for ii in range(0,Data_Number):
        a_vert=np.zeros((a1_N))
        a_hort=np.zeros((a1_N))
        a_vert[:]=a1[:,1]
        a_hort[:]=a1[:,2]
        a_N=a1_N
    
    amax=max(a_vert)
    amin=min(a_vert)
    a_vert=((a_vert-amin)/(amax-amin))
    amax=max(a_hort)
    amin=min(a_hort)
    a_hort=((a_hort-amin)/(amax-amin))

    f1, t1, S1 = signal.spectrogram(a_vert, fs=freq, window=signal.get_window('hann',win), noverlap=overlap)

    f_size,t_size = S1.shape

    f_vector=np.zeros((t_size))

    f=open('exampleFrequencyTable.dat','w+')
    
    for k in range(0,t_size):
        larger=0.0
        for j in range(0,f_size):
            if (S1[j,k] > larger):
                larger=S1[j,k]
                larger_f=f1[j]
        f.write('%f %f %f\n' %(t1[k],larger_f,larger))
    print (a[i])


In [ ]:
def graphFrequency(frequencyTableList, frequencyTableName):

    df = pd.DataFrame(frequencyTableList)

    fig, ax = plt.subplots()
    ax.plot(df[0], df[1])

    ax.set(xlabel='time (s)', ylabel='freq (Hz)',
            title= "Frequency Plot")
    ax.grid()

    fig.savefig('Figure_'+frequencyTableName+'.png')

    fig.show()

In [ ]:
graphFrequency('exampleFrequencyTable.dat', "exampleFrequencyTable")